# High Throughput & Low Latency Models

## Exercise 1: Lambda Functions
1. Vào **Lambda** -> **Functions** -> **Create function**. Chọn **Runtime** là `Python`.
2. Viết code là:
   ```python
   import json

   def lambda_handler(event, context):
       theevent = float(event)
       return {
           'statusCode': 200,
           'body': json.dumps('Your number plus one is: ' + str(theevent + 1))
       }
   ```
3. Test -> Create test event -> Nhập 1 số bất kì, VD `11` -> **Save** -> **Test**

   <img src="images/aws1.png" width=500>

   Sẽ thấy output là `12`.

## Exercise 2: Autoscaling
1. Vào **SageMaker Studio** -> **JupyterLab** -> Up file `ex2_Simple_Endpoint.ipynb` lên rồi chạy, sẽ deploy 1 Endpoint.

2. Trong SageMaker, vào **Inference** -> **Endpoints** -> Vào endpoint vừa deploy. \
   Trong **Endpoint runtime settings**, chọn variant -> **Configure auto scaling** \
   Đặt **Maximum instance count** là `3`, **Scale in cool down** là `30`, **Scale out cool down** là `30`. \
   Đặt **Tartget value** là `30`, nghĩa là nếu nhận đc (gần như) cùng lúc 30 invocations of endpoint for every instance thì sẽ tạo 1 new instance.
   Ấn **Save**.

3. Giờ test autoscaling. Quay lại file notebook, thêm code sau rồi chạy:
   ```python
   n = 0
   while n < 400:
       print(predictor.predict(testX[data.feature_names]))
       n = n + 1
   ```
   Ở đây ta thực hiện 400 invocations of endpoint gần như cùng lúc.

4. Quay lại bên trong endpoint. Trong **Monitor**, ta thấy CPUUtilization, MemoryUtilization, DiskUtilization vẫn chưa ở mức tối đa, nghĩa là Autoscaling là more than sufficient.

## Exercise 3: Concurrency
1. Vào **Lambda** -> **Functions** -> Vào function đã tạo ở Ex1.
2. Vào **Configuration** -> **Concurrency and Recursion detection**. Trong **Concurrency**, ấn **Edit**.

   <img src="images/aws2.png" width=1000>

3. Chọn **Reserve concurrency**, đặt là `5`, quan sát **Unreserved account concurrency** thay đổi -> **Save** \
   (Nếu **Unreserved account concurrency** bị giới hạn là 10 thì vào **Service Quotas** -> **AWS services** -> **AWS Lambda**. \
    Chọn **Concurrent execution** -> **Request increase at account level**. Đặt **Increase quota value** là `1000` -> **Request**)

   <img src="images/aws3.png" width=1200>

4. Vào **Versions** -> **Publish new version** -> **Publish**

   <img src="images/aws4.png" width=1200>

5. Quay lại **Concurrency** -> **Provisioned concurrency configurations** -> **Add configuration** \
   Chọn **Qualifier type** là `Version` -> Chọn version vừa publish. \
   Ở **ProQuvisioned concurrency**, ta thấy "5 available" chính là Reserve concurrency đã set. Ấn mũi tên tăng lên `3` -> **Save**

   <img src="images/aws5.png" width=1500>

6. Test lại Lambda function -> Xem output, chú ý phần **Billed Duration**.